In [1]:
from sqlalchemy import create_engine
from sqlalchemy import Column,Integer,Float,String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
class TSMC(Base):
    __tablename__ = '損益表'
    id = Column(Integer,primary_key=True)
    年度 = Column(String,index=True)
    股本_億 = Column(Integer)
    財報評分 = Column(Integer)
    收盤 = Column(Float)
    平均 = Column(Float)
    漲跌 = Column(Float)
    漲跌_百分比 = Column(Float)

conn = create_engine('sqlite:///台積電.sqlite',echo=True)
Base.metadata.create_all(conn)

2021-04-06 16:14:11,640 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-06 16:14:11,640 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("損益表")
2021-04-06 16:14:11,640 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-04-06 16:14:11,640 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("損益表")
2021-04-06 16:14:11,640 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-04-06 16:14:11,646 INFO sqlalchemy.engine.Engine 
CREATE TABLE "損益表" (
	id INTEGER NOT NULL, 
	"年度" VARCHAR, 
	"股本_億" INTEGER, 
	"財報評分" INTEGER, 
	"收盤" FLOAT, 
	"平均" FLOAT, 
	"漲跌" FLOAT, 
	"漲跌_百分比" FLOAT, 
	PRIMARY KEY (id)
)


2021-04-06 16:14:11,647 INFO sqlalchemy.engine.Engine [no key 0.00067s] ()
2021-04-06 16:14:11,689 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_損益表_年度" ON "損益表" ("年度")
2021-04-06 16:14:11,690 INFO sqlalchemy.engine.Engine [no key 0.00087s] ()
2021-04-06 16:14:11,694 INFO sqlalchemy.engine.Engine COMMIT


In [2]:
import requests
url = "https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2330"
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}
response = requests.get(url,headers=header)
response.encoding = 'utf-8'

In [3]:
def toFloat(s):
    try:
        return float(s.replace(',',''))
    except:
        return None

def toInt(s):
    try:
        return int(s.replace(',',''))
    except:
        return None
 

In [4]:
import re
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
conn = create_engine('sqlite:///台積電.sqlite',echo=True)
bs = BeautifulSoup(response.text,'lxml')
data = bs.find('div',id='txtFinDetailData').find_all('tr',{'id':re.compile('^row')})
session = Session(conn)
for row in data:
    tsmc = TSMC()
    tdlist = row.find_all('td')
    tsmc.年度 = tdlist[0].string
    tsmc.股本_億 = toInt(tdlist[1].string)
    tsmc.財報評分 = toInt(tdlist[2].string)
    tsmc.收盤 = toFloat(tdlist[3].string)
    tsmc.平均 = toFloat(tdlist[4].string)
    tsmc.漲跌 = toFloat(tdlist[5].string)
    tsmc.漲跌_百分比 = toFloat(tdlist[6].string)
    try:
        session.add(tsmc)
    except:
        session.rollback()
        print("出錯")
session.commit()    

2021-04-06 16:14:12,490 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-06 16:14:12,505 INFO sqlalchemy.engine.Engine INSERT INTO "損益表" ("年度", "股本_億", "財報評分", "收盤", "平均", "漲跌", "漲跌_百分比") VALUES (?, ?, ?, ?, ?, ?, ?)
2021-04-06 16:14:12,505 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ('2021', None, None, 610.0, 608.0, 80.0, 15.1)
2021-04-06 16:14:12,527 INFO sqlalchemy.engine.Engine INSERT INTO "損益表" ("年度", "股本_億", "財報評分", "收盤", "平均", "漲跌", "漲跌_百分比") VALUES (?, ?, ?, ?, ?, ?, ?)
2021-04-06 16:14:12,527 INFO sqlalchemy.engine.Engine [cached since 0.02182s ago] ('2020', 2593, 89, 530.0, 379.0, 199.0, 60.1)
2021-04-06 16:14:12,528 INFO sqlalchemy.engine.Engine INSERT INTO "損益表" ("年度", "股本_億", "財報評分", "收盤", "平均", "漲跌", "漲跌_百分比") VALUES (?, ?, ?, ?, ?, ?, ?)
2021-04-06 16:14:12,529 INFO sqlalchemy.engine.Engine [cached since 0.02347s ago] ('2019', 2593, 89, 331.0, 262.0, 105.5, 46.8)
2021-04-06 16:14:12,530 INFO sqlalchemy.engine.Engine INSERT INTO "損益表" ("年度", "股本_億", "財報評分

2021-04-06 16:14:12,569 INFO sqlalchemy.engine.Engine [cached since 0.06371s ago] ('1992', None, None, None, None, None, None)
2021-04-06 16:14:12,570 INFO sqlalchemy.engine.Engine INSERT INTO "損益表" ("年度", "股本_億", "財報評分", "收盤", "平均", "漲跌", "漲跌_百分比") VALUES (?, ?, ?, ?, ?, ?, ?)
2021-04-06 16:14:12,571 INFO sqlalchemy.engine.Engine [cached since 0.06484s ago] ('1991', None, None, None, None, None, None)
2021-04-06 16:14:12,572 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
#select()
#全選
for tsmc in session.query(TSMC).order_by(TSMC.id):
    print(tsmc.年度)

2021-04-06 16:14:12,585 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-06 16:14:12,587 INFO sqlalchemy.engine.Engine SELECT "損益表".id AS "損益表_id", "損益表"."年度" AS "損益表_年度", "損益表"."股本_億" AS "損益表_股本_億", "損益表"."財報評分" AS "損益表_財報評分", "損益表"."收盤" AS "損益表_收盤", "損益表"."平均" AS "損益表_平均", "損益表"."漲跌" AS "損益表_漲跌", "損益表"."漲跌_百分比" AS "損益表_漲跌_百分比" 
FROM "損益表" ORDER BY "損益表".id
2021-04-06 16:14:12,588 INFO sqlalchemy.engine.Engine [generated in 0.00058s] ()
2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992
1991


In [6]:
#過濾只有一筆
tsmc=session.query(TSMC).filter_by(年度='2020').first()
tsmc.財報評分

2021-04-06 16:14:12,601 INFO sqlalchemy.engine.Engine SELECT "損益表".id AS "損益表_id", "損益表"."年度" AS "損益表_年度", "損益表"."股本_億" AS "損益表_股本_億", "損益表"."財報評分" AS "損益表_財報評分", "損益表"."收盤" AS "損益表_收盤", "損益表"."平均" AS "損益表_平均", "損益表"."漲跌" AS "損益表_漲跌", "損益表"."漲跌_百分比" AS "損益表_漲跌_百分比" 
FROM "損益表" 
WHERE "損益表"."年度" = ?
 LIMIT ? OFFSET ?
2021-04-06 16:14:12,601 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ('2020', 1, 0)


89

In [7]:
#過濾有多筆
tsmcList=session.query(TSMC).order_by(TSMC.年度).filter(TSMC.財報評分>=90).all()
for tsmc in tsmcList:
    print(tsmc.年度)

2021-04-06 16:14:12,616 INFO sqlalchemy.engine.Engine SELECT "損益表".id AS "損益表_id", "損益表"."年度" AS "損益表_年度", "損益表"."股本_億" AS "損益表_股本_億", "損益表"."財報評分" AS "損益表_財報評分", "損益表"."收盤" AS "損益表_收盤", "損益表"."平均" AS "損益表_平均", "損益表"."漲跌" AS "損益表_漲跌", "損益表"."漲跌_百分比" AS "損益表_漲跌_百分比" 
FROM "損益表" 
WHERE "損益表"."財報評分" >= ? ORDER BY "損益表"."年度"
2021-04-06 16:14:12,617 INFO sqlalchemy.engine.Engine [generated in 0.00118s] (90,)
1995
1996
2000
2005
2006
2007
2008
2009
2010
2011
2014
2015
2016
2017
2018


In [8]:
#修改
tsmc=session.query(TSMC).filter_by(年度='2020').first()
tsmc.財報評分 = 90
session.commit()

2021-04-06 16:14:12,625 INFO sqlalchemy.engine.Engine SELECT "損益表".id AS "損益表_id", "損益表"."年度" AS "損益表_年度", "損益表"."股本_億" AS "損益表_股本_億", "損益表"."財報評分" AS "損益表_財報評分", "損益表"."收盤" AS "損益表_收盤", "損益表"."平均" AS "損益表_平均", "損益表"."漲跌" AS "損益表_漲跌", "損益表"."漲跌_百分比" AS "損益表_漲跌_百分比" 
FROM "損益表" 
WHERE "損益表"."年度" = ?
 LIMIT ? OFFSET ?
2021-04-06 16:14:12,628 INFO sqlalchemy.engine.Engine [cached since 0.027s ago] ('2020', 1, 0)
2021-04-06 16:14:12,632 INFO sqlalchemy.engine.Engine UPDATE "損益表" SET "財報評分"=? WHERE "損益表".id = ?
2021-04-06 16:14:12,633 INFO sqlalchemy.engine.Engine [generated in 0.00103s] (90, 2)
2021-04-06 16:14:12,634 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
#刪除
tsmc=session.query(TSMC).filter_by(年度='2020').first()
session.delete(tsmc)
session.commit()

2021-04-06 16:14:12,650 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-06 16:14:12,651 INFO sqlalchemy.engine.Engine SELECT "損益表".id AS "損益表_id", "損益表"."年度" AS "損益表_年度", "損益表"."股本_億" AS "損益表_股本_億", "損益表"."財報評分" AS "損益表_財報評分", "損益表"."收盤" AS "損益表_收盤", "損益表"."平均" AS "損益表_平均", "損益表"."漲跌" AS "損益表_漲跌", "損益表"."漲跌_百分比" AS "損益表_漲跌_百分比" 
FROM "損益表" 
WHERE "損益表"."年度" = ?
 LIMIT ? OFFSET ?
2021-04-06 16:14:12,652 INFO sqlalchemy.engine.Engine [cached since 0.05087s ago] ('2020', 1, 0)
2021-04-06 16:14:12,653 INFO sqlalchemy.engine.Engine DELETE FROM "損益表" WHERE "損益表".id = ?
2021-04-06 16:14:12,654 INFO sqlalchemy.engine.Engine [generated in 0.00080s] (2,)
2021-04-06 16:14:12,655 INFO sqlalchemy.engine.Engine COMMIT
